In [2]:
import pandas as pd
import numpy as np
import re
import string
import sys
from io import StringIO

In [5]:
#df = pd.read_csv(r'../data/GTZAN/names_dataframe_complete.csv')
df = pd.read_csv(r'../data/GTZAN/lyrics_dataframe.csv')
df

,filename,artist,title,genre,lyrics,instrumental
0,blues.00000.wav,John Lee Hooker,"One Bourbon, One Scotch And One Beer",blues,"One bourbon, one scotch and one beer\nOne bour...",False
1,blues.00001.wav,John Lee Hooker,I'm In The Mood For Love,blues,"I'm in the mood baby, I'm in the mood for love...",False
2,blues.00002.wav,John Lee Hooker,Think Twice Before You Go,blues,"\nThink twice before you go, baby, this will b...",False
3,blues.00003.wav,John Lee Hooker,I'm Bad Like Jesse James,blues,"\n...A little thing I'm going to do called ""I'...",False
4,blues.00004.wav,John Lee Hooker,Walkin’ The Boogie,blues,"Well my mama she didn't 'low me, just to stay ...",False
...,...,...,...,...,...,...
970,rock.00095.wav,The Stone Roses,Bye Bye Bad Man,rock,\nSoak me to my skin\nWill you drown me in you...,False
971,rock.00096.wav,The Stone Roses,(Song For My) Sugar Spun Sister,rock,"\nHer hair, soft drifted snow\nDeath white, I'...",False
972,rock.00097.wav,The Stone Roses,Made Of Stone,rock,\nYour knuckles whiten on the wheel\nThe last ...,False
973,rock.00098.wav,The Stone Roses,Shoot You Down,rock,You know it\nYou show it\nAnd the time has com...,False


In [6]:
print(set(df.genre))

{'rock', 'blues', 'jazz', 'metal', 'hiphop', 'country', 'classical', 'disco', 'pop', 'reggae'}


In [7]:
print(df[df['title'].isna()])
indices = df[df['title'].isna()].index.sort_values(ascending=False)

for i in indices:
  df.drop(index=i, inplace=True)

df[df['title'].isna()]

Empty DataFrame
Columns: [filename, artist, title, genre, lyrics, instrumental]
Index: []


,filename,artist,title,genre,lyrics,instrumental


In [8]:
# genre counts of all identified songs
df['genre'].value_counts()

blues        100
disco        100
pop          100
reggae       100
rock         100
country       99
hiphop        99
metal         98
jazz          90
classical     89
Name: genre, dtype: int64

## Testing out different APIs for getting lyrics

In [120]:
from musixmatch import Musixmatch
import musixmatch_api_key

musixmatch = Musixmatch(musixmatch_api_key.client_access_token)

In [121]:
song = musixmatch.matcher_lyrics_get(q_track = "Final Countdown", q_artist = "Europe")
# free API key only gives 30% of lyrics
print(song['message']['body']['lyrics']['lyrics_body'])

We're leaving together
But still it's farewell
And maybe we'll come back to Earth, who can tell?

I guess there is no one to blame
We're leaving ground (leaving ground)
Will things ever be the same again?

It's the final countdown

The final countdown

...

******* This Lyrics is NOT for Commercial use *******
(1409622722826)


In [111]:
import azapi

API = azapi.AZlyrics()

API.artist = "Europe"
API.title = "Final Countdown"
API.getLyrics(save=False)

print(API.lyrics)

Failed to find lyrics. Trying to get link from Google
Ten, nine, eight, seven, six, five, four, three, two, one.

We're leaving together,
But still it's farewell.
And maybe we'll come back
To earth, who can tell?

I guess there is no one to blame.
We're leaving ground (leaving ground).
Will things ever be the same again?

It's the final countdown.
The final countdown.

We're heading for Venus (Venus).
And still we stand tall.
'Cause maybe they've seen us.
And welcome us all, yeah.

With so many light years to go
And things to be found (to be found)
I'm sure that we'll all miss her so.

It's the final countdown.
The final countdown.
The final countdown.

The final countdown.

It's the final countdown.
The final countdown.
The final countdown.

It's the final countdown.
We're leaving together.
The final countdown.
We'll all miss her so.
It's the final countdown.

It's the final countdown.


In [112]:
from lyricsgenius import Genius
import genius_api_key

genius = Genius(genius_api_key.client_access_token)

In [118]:
genius.timeout = 10
genius.sleep_time = 1

In [67]:
def clean_lyrics(lyrics):
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    
    # remove header
    index = lyrics.find('Lyrics')
    if index > -1:
        lyrics = lyrics[index+len('Lyrics'):]

    # remove end
    index = lyrics.find('Embed')
    if index > -1:
        lyrics = lyrics[:index]
    lyrics = lyrics.rstrip(string.digits)
    
    return lyrics

In [119]:
song = genius.search_song(artist="Europe", title="Final Countdown")
lyrics = clean_lyrics(song.lyrics)
print(lyrics)

Searching for "Final Countdown" by Europe...
Done.

We're leaving together, but still, it's farewell
And maybe, we'll come back to Earth, who can tell?
I guess there is no one to blame
We're leaving ground 
Will things ever be the same again?


It's the final countdown
The final countdown, oh

We're heading for Venus 
And still, we stand tall
'Cause maybe they've seen us
And welcome us all, yeah
With so many lightyears to go
And things to be found 
I'm sure that we'll all miss her, so


It's the final countdown
The final countdown
The final countdown , oh




The final countdown
Oh, it's the final countdown
The final countdown
The final countdown 
Oh, it's the final countdown
We're leaving together 
We'll all miss her so
It's the final countdown 

Oh, it's the final countdown
Yeah


## Scrape lyrics for all songs

In [14]:
for idx in df.index:
    song = genius.search_song(artist=df.artist[idx], title=df.title[idx])
    if song:
        lyrics = clean_lyrics(song.lyrics)
        #print(lyrics)
        df.loc[idx, 'lyrics'] = lyrics
    #else:
    #    print("Song not found")

Searching for "One Bourbon, One Scotch And One Beer" by John Lee Hooker...
Done.
Searching for "I'm In The Mood For Love" by John Lee Hooker...
Done.
Searching for "Think Twice Before You Go" by John Lee Hooker...
Done.
Searching for "I'm Bad Like Jesse James" by John Lee Hooker...
Done.
Searching for "Walkin’ The Boogie" by John Lee Hooker...
Done.
Searching for "Sugar Mama" by John Lee Hooker...
Done.
Searching for "Dreamin' Blues" by John Lee Hooker...
Done.
Searching for "Ground Hog Blues" by John Lee Hooker...
Done.
Searching for "Shake It Baby" by John Lee Hooker...
Done.
Searching for "It's My Own Fault" by John Lee Hooker...
Done.
Searching for "Lonely Boy Boogie" by John Lee Hooker...
Done.
Searching for "It Serves Me Right To Suffer" by John Lee Hooker...
Done.
Searching for "Cross Road Blues" by Robert Johnson...
Done.
Searching for "Terraplane Blues" by Robert Johnson...
Done.
Searching for "Come On In My Kitchen" by Robert Johnson...
Done.
Searching for "Walking blues" by 

Timeout: Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)

In [72]:
for idx in df[df.lyrics.isna()].index:
    stdout = sys.stdout
    s = StringIO()
    sys.stdout = s
    song = genius.search_song(artist=df.artist[idx], title=df.title[idx])
    sys.stdout = stdout
    s.seek(0)
    logs = s.getvalue()
    if song:
        lyrics = clean_lyrics(song.lyrics)
        print("Found lyrics!")
        df.loc[idx, 'lyrics'] = lyrics
    else:
        if "Specified song does not contain lyrics." in logs:
            df.loc[idx, 'instrumental'] = True
            print("Song does not have lyrics.")
        elif "No results found for:" in logs:
            print("Song not found.")
        else:
            print("Other error.")

In [75]:
# for which songs no lyrics were found
df[df.lyrics.isna()].genre.value_counts()

classical    81
jazz         72
blues        22
reggae        4
country       3
metal         2
disco         1
hiphop        1
Name: genre, dtype: int64

In [74]:
from operator import index

df.to_csv('../data/GTZAN/lyrics_dataframe.csv', index=False)